In [24]:
from bs4 import BeautifulSoup
import re
import pandas as pd

In [27]:

def scrape_jobs(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    jobs = []
    
    # Encontrar todos los elementos de trabajo
    job_cards = soup.find_all('div', class_='cardOutline')
    
    for job in job_cards:
        job_data = {}

        # Obtener toda la información dentro de un div con un conjunto de clases especificas
        div_info = job.find('div', class_='jobsearch-JobComponent-description')
        if div_info:
            job_data['div_info'] = div_info.get_text(strip=True)

        # Título del trabajo
        title_elem = job.find('h2', class_='jobTitle')
        if title_elem:
            job_data['title'] = title_elem.get_text(strip=True)
        
        # Nombre de la empresa
        company_elem = job.find('span', attrs={'data-testid': 'company-name'})
        if company_elem:
            job_data['company'] = company_elem.get_text(strip=True)
        
        # Ubicación
        location_elem = job.find('div', attrs={'data-testid': 'text-location'})
        if location_elem:
            job_data['location'] = location_elem.get_text(strip=True)
        
        # Salario
        salary_elem = job.find('div', attrs={'data-testid': 'attribute_snippet_testid'}, 
                             string=re.compile(r'\$'))
        if salary_elem:
            job_data['salary'] = salary_elem.get_text(strip=True)
        else:
            job_data['salary'] = "No especificado"
        
        # Requisitos (extraídos del snippet)
        requirements = []
        snippet_elem = job.find('div', attrs={'data-testid': 'belowJobSnippet'})
        if snippet_elem:
            for li in snippet_elem.find_all('li'):
                requirements.append(li.get_text(strip=True))
        job_data['requirements'] = requirements if requirements else ["No especificados"]
        
        # Enlace del trabajo
        link_elem = job.find('a', class_='jcs-JobTitle')
        if link_elem and 'href' in link_elem.attrs:
            job_data['link'] = "https://mx.indeed.com" + link_elem['href'] if not link_elem['href'].startswith('http') else link_elem['href']
        
        jobs.append(job_data)
    
    return jobs

# Ejemplo de uso (puedes reemplazar esto con tu código HTML)
if __name__ == "__main__":
    # Aquí puedes pegar tu código HTML o leerlo desde un archivo
    with open('indeed_jobs.html', 'r', encoding='utf-8') as file:
        html_content = file.read()
    
    jobs = scrape_jobs(html_content)
    
    # Imprimir resultados
    for i, job in enumerate(jobs, 1):
        print(f"\nTrabajo #{i}")
        print(f"Título: {job.get('title', 'N/A')}")
        print(f"Empresa: {job.get('company', 'N/A')}")
        print(f"Ubicación: {job.get('location', 'N/A')}")
        print(f"Salario: {job.get('salary', 'N/A')}")
        print("Requisitos destacados:")
        for req in job.get('requirements', []):
            print(f"- {req}")
        print(f"Enlace: {job.get('link', 'N/A')}")
        print(f"Información adicional: {job.get('div_info', 'N/A')}")


Trabajo #1
Título: Ingeniero de Soporte Técnico
Empresa: Confidencial
Ubicación: Ciudad Juárez, Chih.
Salario: $20,000 a $21,000 por mes
Requisitos destacados:
- En Soporte técnico ensoftwarey hardware.
- Administración y control de inventarios de equipo.
- Mantenimiento preventivo y correctivo.
Enlace: https://mx.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0CCuAlW1XOuHrwuDoESYE5WiC8F82eUoqkwZs8tBnJjleS8bAh5ZWV1CcE0MlLXKXReFUFwtPjCmFJJLY3SQYBwIbEe8gIEj9yBOnXB5wgCEEszVSR-zOX_BZJxlFaURp0ehco-xa2H7kFatsj7XH14kfWWf1l6TL7UFAsvFJTB-npntp_YntrhHKlyrLylZ00VFXNlbI0ZYu1UPH7kMpNwuMcGQg_qNvUdOisUUirmIukDGuigCYCxr7Rb41VvrqYB64_NPOr9i1ppLVtuBwmur45wq1PPIQOxbj8QnH85bXM19IN6E61qNpKznsVgwVcAbhe08D7Zd--jjlMi5Mv2ZfVge6Z1_F7b3gBA0qUnYvVeacveGfgnIqE9EaGY1kleBjMJL4wUhehQyz3dRndoFpuzKTAvjesoUhkxjgVhgee8XCIxs-uYFrv3jIUXB7ckMIOd0R22LJHB-Vy0JrdttGRLVspMXlVUfIsBWwfWBu0cfj3hDz2qA7G9faoNnCF_dJpyKYN1xxjTclxa5_qRuCgfzrFE7OPetNJxEbK8dBgr9oN3gSY_z65J4yCfwo1rndyp1Tr0d9n2XmO9roRnvu5GsMVEzhrZk2Kyg_CA8tOPSJhxIZnnvCKUZcKOmbWkI

In [26]:
df = pd.DataFrame(jobs)
df

,title,company,location,salary,requirements,link
0,Ingeniero de Soporte Técnico,Confidencial,"Ciudad Juárez, Chih.","$20,000 a $21,000 por mes","[En Soporte técnico ensoftwarey hardware., Adm...",https://mx.indeed.com/pagead/clk?mo=r&ad=-6NYl...
1,DESARROLLADOR DE APLICACIONES,Border Express de México,"Ciudad Juárez, Chih.","$28,000 a $30,000 por mes","[Lenguajes de programación: JavaScript, Python...",https://mx.indeed.com/pagead/clk?mo=r&ad=-6NYl...
2,Software Developer (Medical Industry),BEPC Inc,"32560, Jardines de San Pablo, Chih.",No especificado,[Participate in technical design and documenta...,https://mx.indeed.com/pagead/clk?mo=r&ad=-6NYl...
3,Desarrollador de Software,Grupo Ei,"Ciudad Juárez, Chih.","$22,600 a $30,660 por mes","[Desarrollar proyectos y mejoras internas., In...",https://mx.indeed.com/rc/clk?jk=24a8b13b9240d8...
4,Especialista de IT,Grupo TECMA,"Ciudad Juárez, Chih.",No especificado,[Amplio criterio en solución de soporte enfoca...,https://mx.indeed.com/rc/clk?jk=a729a3999fd09d...
5,Consultor de Software,Grupo Tress Internacional,"Ciudad Juárez, Chih.",No especificado,"[Configurar, implementar y brindar capacitació...",https://mx.indeed.com/rc/clk?jk=66d39a1b34d8f9...
6,Workplace DevOps Engineer,Apexon,México,No especificado,"[Identify and address performance bottlenecks,...",https://mx.indeed.com/pagead/clk?mo=r&ad=-6NYl...
7,Consultor Jr de ERP (Soporte Técnico),Grupo Tress Internacional,"Ciudad Juárez, Chih.",No especificado,[Aprenderás a fondo sobre nuestrosoftwarey lo ...,https://mx.indeed.com/rc/clk?jk=88e4ef28476be9...
8,Soporte Técnico,Pagos Digitales,"Ciudad Juárez, Chih.","$12,000 a $15,000 por mes",[Reparar o reemplazar dispositivos de cajeros ...,https://mx.indeed.com/rc/clk?jk=f08d75b40c44b6...
9,Practicante base de datos,BRP,"Ciudad Juárez, Chih.",No especificado,[BRP es líder mundial en vehículos recreativos...,https://mx.indeed.com/rc/clk?jk=0b9dfa18fd7b62...


In [32]:
from bs4 import BeautifulSoup

def obtener_texto_div_principal(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    trabajos = []
    # Encuentra todos los divs principales de cada trabajo
    for div in soup.find_all('div', class_='jobsearch-JobComponent-description css-1rybqxq eu4oa1w0'):
        texto = div.get_text(separator='\n', strip=True)
        trabajos.append(texto)
    return trabajos

# Ejemplo de uso:
with open('indeed_jobs.html', 'r', encoding='utf-8') as file:
    html_content = file.read()

todos_los_textos = obtener_texto_div_principal(html_content)
for i, texto in enumerate(todos_los_textos, 1):
    print(f"\nTrabajo #{i}\n{texto}\n")


Trabajo #1
ESCOLARIDAD: Ingenieria en Sistemas o afin
EXPERIENCIA: 1 año en posición similar.
ACTIVIDADES
1. En Soporte técnico en software y hardware.
2. Administración y control de inventarios de equipo.
3. Monitoreo de equipos de impresión en sucursales, reemplazos de toner y seguimiento a su entrega a instalación.
4. Atención a tickets
5. Mantenimiento preventivo y correctivo
Tipo de puesto: Tiempo completo
Sueldo: $20,000.00 - $21,000.00 al mes
Pago complementario:
Bono de productividad
Tipo de jornada:
Incluye fines de semana
Turno de 10 horas
Escolaridad:
Licenciatura terminada (Obligatorio)
Experiencia:
Soporte técnico: 1 año (Obligatorio)
Lugar de trabajo: Empleo presencial

